In [3]:
import numpy as np
import os, time, sys
import time
import importlib as imp
import matplotlib.pyplot as plt
print("Python version: " + sys.version)
lumapi = imp.machinery.SourceFileLoader("lumepi","D:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py").load_module()

Python version: 3.7.6 (tags/v3.7.6:43364a7ae0, Dec 19 2019, 00:42:30) [MSC v.1916 64 bit (AMD64)]


In [4]:
hight_map = np.load('./height_map.npy')
# hight_map

##### Helper function
```ruby
dir(fdtd)    
help(fdtd.getname)   
fdtd = lumapi.FDTD(filename = "D2NN_simulation.fsp")    
fdtd.close() 
```


In [5]:
num_layer = 5          # number of the layers
size = 5              # size of the network

x_span = 400e-6        # pixel size: 400 um (400e-6*200 = 0.08)
y_span = 400e-6
z = 3e-2               # distance of propagation(the distance bewteen two layers)

material = "<Object Defined Dielectric>"


fdtd = lumapi.FDTD(hide = False)   # hide the GUI of the software


def override_layer_mesh(layer_name, layer_z_position):
    fdtd.groupscope("::model")
    fdtd.addmesh()
    fdtd.set("name",layer_name + "mesh")
    # set dimension
    fdtd.set("x",size*x_span/2)
    fdtd.set('x span',size*x_span)      
    fdtd.set("y",-1*size*y_span/2)    
    fdtd.set("y span",size*x_span)
    fdtd.set("z",layer_z_position)
    fdtd.set("z span",1.2e-5)                         # 这个值是height_map里面最厚的那个值                            
    # enable in X,Y,Z direction
    # 1:enable, 0:disable
    fdtd.set("override x mesh",0)
    fdtd.set("override y mesh",0)
    fdtd.set("override z mesh",0)
    # restrict mesh by defining maximum step size
    fdtd.set("set maximum mesh step",1)
    fdtd.set("dx",100e-6)
    fdtd.set("dy",100e-6)
    fdtd.set("dz",100e-6)
    return None

# add diffraction layers
for l in range(0,num_layer):                                # l is the layer number
    fdtd.addstructuregroup()
    groupname = "diffraction_layer_"+ str(l)                # one group represents one layer
    fdtd.set("name",groupname)
    
    fdtd.groupscope(groupname)
    layer_z_position = 0 + l*z
    for j in range(0,size):                                 # i:x-axis, j: y-axis
        for i in range(0,size):
            fdtd.addrect()                                  # create and select the rect
            fdtd.set("name","rect"+str(i)+"_"+str(j))       # set the name of the little rect
            fdtd.set("x",x_span/2 + i*x_span)               # set the x center position of the rect
            fdtd.set("x span",x_span)
            fdtd.set("y",-y_span/2 - j*y_span)              # set the y center position of the rect
            fdtd.set("y span",y_span)  
            fdtd.set("z",0 + layer_z_position)              # set the z center position of the rect
            fdtd.set("z span",hight_map[l][j][i])
            fdtd.set("material",material)
            # fdtd.addtogroup(groupname)
    fdtd.groupscope("::model")    # after create one layer, change the group scope back to the model
    override_layer_mesh(layer_name = groupname, layer_z_position = layer_z_position)
    
# add a simulation region
fdtd.groupscope("::model")
fdtd.addfdtd()               
fdtd.set('dimension',2)   # 1 = 2D, 2 = 3D, set a 3D simulation region
fdtd.set("x",size*x_span/2)
fdtd.set('x span',size*x_span)      # set the x span of the simulation region
fdtd.set("y",-1*size*y_span/2)        # we are using the negative part of the y-axis
fdtd.set("y span",size*x_span)
fdtd.set("z",5*z/2)
fdtd.set("z span",5*z)

# add monitor
fdtd.addpower()
fdtd.set('name','detector_plane_monitor')
fdtd.set("x",size*x_span/2)
fdtd.set('x span',size*x_span)      
fdtd.set("y",-1*size*y_span/2)    
fdtd.set("y span",size*x_span)
fdtd.set("z",5*z/2)


fdtd.save("D2NN_Simulation.fsp")

# run simulation
# fdtd.run()

In [5]:
# get result and plot
T = fdtd.getresult('monitor','time signal')
S = fdtd.getresult('monitor','spectrum')

In [12]:
fdtd.visualize(T) # 在软件里可视化

In [12]:
# print(type(S))
# print(S)
# plt.plot(S['lambda'])
# T['time']